In [1]:
from pyspark.sql import SparkSession


In [2]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [5]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [6]:
lr = LinearRegression(featuresCol="features",labelCol='label',predictionCol='prediction')

In [7]:
lrModel = lr.fit(training)

In [9]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [10]:
lrModel.intercept

0.14228558260358093

In [11]:
training_summary = lrModel.summary

In [13]:
training_summary.rootMeanSquaredError

10.16309157133015

In [14]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [15]:
split_object = all_data.randomSplit([0.7,0.3])

In [16]:
split_object

[DataFrame[label: double, features: vector],
 DataFrame[label: double, features: vector]]

In [17]:
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [18]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-23.487440120936512|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|
|-17.026492264209548|(10,[0,1,2,3,4,5,...|
+----------

In [19]:
correct_model = lr.fit(train_data)

In [20]:
test_results = correct_model.evaluate(test_data)

In [21]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -25.18928469523095|
| -27.12722330746525|
| -20.59529275353972|
|-18.775541456978722|
| -16.70962849479044|
|-15.750328855890867|
| -17.23462773600546|
|-12.255185244889553|
|-11.380672753090142|
|-16.993002363791753|
| -9.868312980070987|
|-18.186733435342294|
|-11.526952408401964|
|-14.218819905541228|
| -15.63275549706336|
| -4.886773844464667|
| -14.84123967517466|
|-11.796118932653023|
| -8.157048875516125|
| -8.602619888472994|
+-------------------+
only showing top 20 rows



In [23]:
test_results.rootMeanSquaredError

10.9795396070193

In [24]:
unlabeled_data = test_data.select('features')

In [25]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [26]:
predictions = correct_model.transform(unlabeled_data)

In [27]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...| -2.856733342545683|
|(10,[0,1,2,3,4,5,...|  4.177397371269175|
|(10,[0,1,2,3,4,5,...| 0.3832154945810469|
|(10,[0,1,2,3,4,5,...|-0.8917771583929965|
|(10,[0,1,2,3,4,5,...|-0.7845718620929045|
|(10,[0,1,2,3,4,5,...|-0.9687679777142213|
|(10,[0,1,2,3,4,5,...| 1.8750828561727826|
|(10,[0,1,2,3,4,5,...| -3.093685910489701|
|(10,[0,1,2,3,4,5,...|-2.4864151420686254|
|(10,[0,1,2,3,4,5,...| 3.2205608020888823|
|(10,[0,1,2,3,4,5,...|-3.5522817958197703|
|(10,[0,1,2,3,4,5,...|  5.264510331971873|
|(10,[0,1,2,3,4,5,...|-0.9523278030495321|
|(10,[0,1,2,3,4,5,...|  2.340652405574387|
|(10,[0,1,2,3,4,5,...|  3.805682500670789|
|(10,[0,1,2,3,4,5,...| -6.753775833424159|
|(10,[0,1,2,3,4,5,...|  3.409437309634199|
|(10,[0,1,2,3,4,5,...| 0.8501992748700902|
|(10,[0,1,2,3,4,5,...|-2.1316083768725833|
|(10,[0,1,2,3,4,5,...|-1.5855374867181171|
+----------